In [1]:
# 더보기 계속 클릭하기
def clickMore(driver):
    while True:
        try:
            more_button = driver.find_element_by_css_selector('#comment > div.ft > a')
            more_button.click()
            time.sleep(1)
        except:
            break

In [2]:
# 한 기사의 댓글 추출
def getComment(driver,keyword):
    
    import os
    import pandas as pd
    
    # 데이터를 담을 딕셔너리
    comment_dict = {
        '제목' : [],
        '날짜' : [],
        '작성일' : [],
        '작성자':[],
        '댓글' : []
    }
    
    # 기사 제목 태그
    a0 = driver.find_elements_by_css_selector('#article_title')
    # print(a0)
    
    if not a0:
        print('댓글없음')
        return False
    
    else:
        # 기사 제목 태그
        data1 = a0[0].text
        # print(data1)
        
        # 날짜
        data2 = driver.find_elements_by_css_selector('div.article_head > div > div.byline')[0].text
        # print(data2)
    
        # 댓글 갯수 확인
        a0 = driver.find_element_by_css_selector('#comment > div.hd > strong > span').text
        # print(a0)   

        # 댓글 수가 0이면
        if a0 == '0':
            print('댓글 없음')
            return False
        else:
            # 댓글이 담긴 ul 태그 들고오기 
            a1 = driver.find_elements_by_css_selector('#comment > div.bd > div.comment_list > ul > li')
            # print(a1)

            for a2 in a1:

                a22 = a2.find_elements_by_css_selector('div')

                # 제목 & 날짜 저장
                comment_dict['제목'].append(data1) 
                comment_dict['날짜'].append(data2) 

                # 댓글 작성일
                if a2.find_elements_by_css_selector('div > div.cmt_area > dl > dt > span'):
                    a3 = a2.find_elements_by_css_selector('div > div.cmt_area > dl > dt > span')[0].text
                    comment_dict['작성일'].append(a3)
                else:
                    pass

                # 댓글 작성자가 있으면 가져오기
                writer = a2.find_elements_by_css_selector('div > div.cmt_area > dl > dt > strong')
                if writer:
                    writer = writer[0].text
                    comment_dict['작성자'].append(writer)
                else:
                    comment_dict['작성자'] = np.nan
                print(writer)

                # 댓글 
                a4 = a2.find_elements_by_css_selector('div > div.cmt_area > dl > dd > p')[0].text
                comment_dict['댓글'].append(a4)
                # print(a4)

                # 댓글에 댓글이 있는 경우
                if len(a22) > 5 :

                    # 댓글의 댓글이 담긴 div
                    a5 = a2.find_elements_by_css_selector('div.reply_area > ul > li')
                    # print(len(a5))

                    for a6 in a5:

                        # 제목 & 날짜
                        comment_dict['제목'].append(data1) 
                        comment_dict['날짜'].append(data2) 

                        # 댓글작성일
                        a7 = a6.find_elements_by_css_selector('div > dl > dt > span')[0].text
                        # print(a7)
                        comment_dict['작성일'].append(a7)

                        re_writer = a6.find_elements_by_css_selector(
                                    'div > dl > dt > strong')[0].text
                        comment_dict['작성자'].append(re_writer)

                        # 댓글
                        a8 = a6.find_elements_by_css_selector('div > dl > dd > p')[0].text
                        # print(a8)
                        comment_dict['댓글'].append(a8)

            df1 = pd.DataFrame(comment_dict)
            df1['날짜'] = [date.split('입력')[-1].split('수정')[0].split('|')[0].strip() \
                           for date in df1['날짜']]
            df1['날짜'] = pd.to_datetime(df1['날짜'])
            df1['작성일'] = pd.to_datetime(df1['작성일'])

            display(df1)

            FILENAME = f'joongang_comment_{keyword}.csv'
            if os.path.exists(FILENAME) == False:
                # 파일이 없을 경우
                df1.to_csv(FILENAME, encoding='utf-8-sig', index=False)
            else:
                # mode='a' : 기존 것 뒤에다 붙여줌
                df1.to_csv(FILENAME, encoding='utf-8-sig', index=False, header=False, mode='a')

            return True

In [3]:
# 뉴스 댓글 가져오는 함수
def getJoongangComment(link_df, keyword, idx=45):
    
    import pandas as pd
    from selenium import webdriver as wd
    from IPython.display import clear_output
    
    num_link = link_df.shape[0]
    # print(link_df.loc[0])

    # 웹 드라이버
    driver = wd.Chrome('./chromedriver.exe')
    idx = 0

    while True :

        driver.implicitly_wait(20)

        clear_output(wait=True)

        driver.get(link_df.loc[idx].values[0])
        clickMore(driver)

        print(f'{idx}번째 기사 댓글 수집 중')

        chk = getComment(driver, keyword)
        # print(f'chk : {chk}')
        
        # 기사에 댓글 데이터 정상 수집
        if chk:
            print(f'{idx}번째 기사 댓글 정상 수집 완료')
        # 기사에 댓글 데이터가 없으면 다음 페이지로 
        else:
            print(f'{idx}번째 기사 댓글 없음, 다음 기사로~')

        idx = idx + 1

        if idx == num_link :
            break

    print('수집완료')

In [4]:
test = False
if test:
    import pandas as pd
    from selenium import webdriver as wd
    from IPython.display import clear_output

    df = pd.read_csv('../중앙일보_link.csv')

    driver = wd.Chrome('./chromedriver.exe')
    idx = 0



    driver.implicitly_wait(20)

    clear_output(wait=True)

    driver.get(df.loc[idx].values[0])
    clickMore(driver)
    getComment(driver)

In [ ]:
import pandas as pd
names = ['정세균','유승민', '이낙연', '홍준표', '안철수', '윤석열', '이재명']
for name in names:
    keyword = name
    link_df = pd.read_csv(f'중앙일보_대선 {keyword}_link.csv')
    getJoongangComment(link_df, keyword)

25번째 기사 댓글 수집 중
bcg7****
서****
kaps****
이****
Chih****
빈****
edoo****
즐거운 ****
esha****
kore****
전****
Cha ****
tiam****
tryb****
ojy0****
전****
1962****
뭄****
rawm****


,제목,날짜,작성일,작성자,댓글
0,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-26 12:19:18,bcg7****,"닥치는대로 다 해쳐먹는 정신 존경합니다 3,000배 하시고 가세요"
1,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-26 08:02:01,서****,노무현 무덤이라는 돌부터 거두어 내라'''귀신이 답답하단다'무식한것들이보기좋다고 돌...
2,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-26 00:45:14,kaps****,간길에 뛰어 내리세요
3,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-25 22:19:22,이****,고마해라 지겹다
4,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-25 22:00:49,Chih****,노무현정신 은 ..돈주고 매매 가능한지요????전세도 주는지?????
5,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-25 21:38:55,빈****,노무현 개18시기넘과 김대중 뻘겅이 18시기넘의 시체를 꺼내어서 부관참수를 하여야 ...
6,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-25 18:56:28,edoo****,"자살 영웅한테 협박을 하면 당선 된다. 총리 때 하던 실력으로,"
7,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-25 18:46:03,즐거운 ****,늙은 쥐가 독 뚫는 꾀만 가젔다 고 했다. 여당 대표. 국회의장. 국무 총리 해 먹...
8,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-25 16:41:59,esha****,Virus 정! 정신 차려라
9,봉하마을 간 정세균 “노무현 미완의 꿈 완성하겠다”,2021-04-25 14:09:00,2021-04-25 15:49:27,kore****,지 주제를 알아야지


25번째 기사 댓글 정상 수집 완료


## 윤석열
45번째 기사에서 댓글 2개 겹침
    - 양****
    - Jhee****